In [ ]:
import os
import numpy as np

In [ ]:
test_file = 'test.1_all.hdf5_fac1.0.npz'
#test_file = 'test.1_all.hdf5_base.npz'
#test_file = 'dev.1_all.hdf5_base.npz'
test_path = './orig/omerlevy-bidaf_no_answer-2e9868b224e4/relation_splits/'

data = np.load( os.path.join(test_path,test_file) )
data.files  # ['predictions', 'targets']

In [ ]:
preds, targs = data['predictions'], data['targets']
preds.shape, targs.shape  # ((10, 1, 5, 128), (10, 1, 128))

In [ ]:
def assess(pred, targ):
    #print(pred.shape, targ.shape)  # (5, 128) (128,)
    
    #for i,v in enumerate(list(targ)):
    #    if v==0: continue
    #    print(i, v)  # 1->2 is subject, 3->4 is required answer(s)
        
    #  Get the starts and ends from targ
    targ_q_starts = [i for i,v in enumerate(list(targ)) if v==1]
    targ_a_starts = [i for i,v in enumerate(list(targ)) if v==3]
    #print(targ_ans_starts)
    
    targ_q_arr, targ_a_arr = [],[]
    for i in targ_q_starts:                    # find next '2'
        # problem at idx=1218 ??
        if 2 in list(targ[i:]):
            targ_q_arr.append( (i, i+list(targ[i:]).index(2) ) )
    for i in targ_a_starts:                    # find next '4'
        # problem at idx=4480 ??
        if 4 in list(targ[i:]):
            targ_a_arr.append( (i, i+list(targ[i:]).index(4) ) )
    print( 'targets : q=', targ_q_arr, ', a=', targ_a_arr )
    
    if False:
        for i,v in enumerate(list(pred.T)):
            print(i, ["%+7.4f" % f for f in list(v)])
            
    # This is pretty 'audacious', since it wasn't trained to succeed at doing this
    print("                      answer: ", np.argmax(pred[3,:]), np.argmax(pred[4,:]),  )
    
    if False:
        for i,v in enumerate(list(pred.T)):
            c = np.argmax(v)
            if c==0: continue
            print(i, c)  # 1->2 is subject, 3->4 is required answer(s)

    
    """
    Evaluation Metrics : Each instance is evaluated by comparing 
    the tokens in the labeled answer set with those of the predicted span. 
    Precision is the true positive count divided by the number of times the system returned a non-null answer. 
    Recall is the true positive count divided by the number of instances that have an answer.
    """
    """  NOT HANDLED
    We ignore word order, case, punctuation, and articles (“a”, “an”, “the”). 
    We also ignore “and”, which often appears when a single span captures multiple correct answers
    (e.g. “United States and Canada”).    
    """
    
    have_ans = 1 if len(targ_a_arr)>0 else 0
    
    tp=0.
    non_null=0

    # Let's see if this naive method works (at all)
    a_start_best = np.argmax(pred[3,:])
    a_end_best   = np.argmax(pred[4,:])
    if a_start_best<a_end_best: # This appears to be a valid guess
        non_null=1 # The system thinks it found something
        
        # Work out the overlap between (a_start_best,a_end_best) and (targ_a_arr)
        #targ_mask = np.zeros(targ.shape)
        #for (i,j) in targ_a_arr:
        #    for k in range(i,j+1):
        #        targ_mask[k]=1
        #targ
        
        # Work out the overlap between (a_start_best,a_end_best) and each element of (targ_a_arr)
        best_overlap=0.
        a0, a1 = a_start_best, a_end_best
        for (b0, b1) in targ_a_arr:
            intersection = min(a1, b1) - max(a0, b0)
            union        = max(a1, b1) - min(a0, b0)
            overlap_fraction = intersection/float(union)
            
            if overlap_fraction>0. and best_overlap<overlap_fraction:
                best_overlap=overlap_fraction
        tp = best_overlap
        
    else:  # Could we have guessed some other way?
        pass
    
    return tp, non_null, have_ans

# idx=0 should be a None, idx=1 should have an Answer, 6 seems correct
idx=6; assess( preds[idx, 0, :, :], targs[idx, 0, :] )

In [ ]:
tp_tot, non_null_tot, have_ans_tot = 0.,0,0
for idx in range(preds.shape[0]):
    #print("idx=", idx)
    tp, non_null, have_ans = assess( preds[idx, 0, :, :], targs[idx, 0, :] )
    print("       #%d assessment :tp=%.1f, non_null=%1d, have_ans=%1d" % (idx, tp, non_null, have_ans,) )
    tp_tot += tp
    non_null_tot += non_null
    have_ans_tot += have_ans

precis = tp_tot / non_null_tot
recall = tp_tot / have_ans_tot

f1 = 2. * (precis*recall)/(precis+recall)

print("precision=%.2f%% recall=%.2f%% F1=%.2f%%" % (100.*precis, 100.*recall, 100.*f1))

In [ ]:
#Paper (BEAT THIS!)
# precision=43.61% recall=36.45% F1=39.61%

#test.1_all.hdf5_base.npz 
# precision=30.97% recall=33.06% F1=31.98%

#test.1_all.hdf5_fac1.0.npz 
# precision=24.67% recall=43.13% F1=31.39%